# Préprocessing table légume (issue du scrapping) et ajout des données agribalyse

## Préprocessing table légume issue du scrapping

### Import et traitement des colonnes nom_légume et origine

In [83]:
import pandas as pd

df_legume = pd.read_csv("legumes_data_2_carrefour.csv")

In [84]:
pd.set_option('display.max_columns', None)

In [49]:
df_legume["nom_legume"] = df_legume["Nom"].str.split(" ").str[1]

In [50]:
df_legume["nom_legume"].unique()

array(['Avocat', 'Asperges', 'Asperge', 'Artichaut', 'Brocoli',
       'Brocolis', 'Coco', 'Epinards', 'Fenouil', 'Haricot', 'Haricots',
       'Jeunes', 'Mini', 'Pois', 'Pousse', 'Blette', 'Blettes', 'Cresson',
       'Endive', 'Endives', 'Feuille', 'Mâche', 'Mélange', 'Pousses',
       'Roquette', 'Salade', 'Salades', 'Sucrine', 'Tomate', 'Tomates',
       'Avocats', 'Concombre', 'Betterave', 'Betteraves', 'Radis',
       'Champignon', 'Champignons', 'Girolles', 'Pleurote', 'Aubergine',
       'Aubergines', 'Courge', 'Courgette', 'Courgettes', 'Légumes',
       'Piment', 'Piments', 'Poivron', 'Poivrons', 'Carotte', 'Carottes',
       'Patate', 'Patates', 'Pomme', 'Pommes', 'Butternut', 'Citrouille',
       'Igname', 'Maïs', 'Manioc', 'Panais', 'Potimarron', 'Potiron',
       'Rutabaga', 'Topinambour', 'Blancs', 'Céleri', 'Chou',
       'Chou-fleur', 'Choux', 'Navet', 'navet', 'Navets', 'Petits',
       'Poireau', 'Poireaux', 'Pot', 'Ail', 'Echalions', 'Echalote',
       'Echalotes', 

In [51]:
df_legume["Origine"] = df_legume["Origine"].str[1:-1]
df_legume["Origine"].unique()

array(['ESPAGNE', 'ISRAEL', 'FRANCE', 'PEROU', 'origine UE et Hors U.E.',
       'ITALIE', 'MAROC', 'RWANDA', 'SENEGAL', 'ORIGINE PAYS TIERS',
       'ORIGINE C.E.E.', 'GUATEMALA', 'UNION EUROPEENNE', 'BELGIQUE',
       'UE ET HORS U.E.', 'REPUBLIQUE DOMINICAINE', 'COLOMBIE',
       'PAYS-BAS', 'POLOGNE', 'CANADA', nan, 'AFRIQUE DU SUD',
       'ORIGINE C.E.E', 'COSTA RICA', 'PORTUGAL', 'LIBAN', 'EGYPTE',
       'ROYAUME-UNI', 'inde\xa0', 'INDONÉSIE', 'SRI LANKA'], dtype=object)

### On ajoute la distance en fonction de la france grâce à geopy -> plus le produit a une origine lointaine, plus son empreinte carbone sera élevée

In [52]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Liste des pays
pays_liste = ['ESPAGNE', 'FRANCE', 'PEROU', 'ITALIE', 'MAROC', 'RWANDA', 'SENEGAL', 
              'GUATEMALA', 'BELGIQUE', 'AFRIQUE DU SUD', 'REPUBLIQUE DOMINICAINE', 
              'ISRAEL', 'COLOMBIE', 'PAYS-BAS', 'POLOGNE', 'CANADA', 'COSTA RICA', 
              'PORTUGAL', 'INDONÉSIE', 'THAÏLANDE', 'inde', 'ROYAUME-UNI', 
              'SRI LANKA', 'EGYPTE']

# Initialiser le géocodeur
geolocator = Nominatim(user_agent="geo_distance_calculator")

# Obtenir les coordonnées de la France
france_location = geolocator.geocode("France")

distances_dict = {}

# Calculer la distance pour chaque pays
for pays in pays_liste:
    pays_name = 'India' if pays.lower() == 'inde' else pays
    
    # Obtenir les coordonnées du pays
    pays_location = geolocator.geocode(pays_name)
    
    # Calculer la distance
    distance = geodesic((france_location.latitude, france_location.longitude),
                        (pays_location.latitude, pays_location.longitude)).kilometers
    
    # Stocker la distance dans le dictionnaire
    distances_dict[pays] = distance

# Afficher le dictionnaire des distances
print(distances_dict)


{'ESPAGNE': 976.2843741584757, 'FRANCE': 0.0, 'PEROU': 9573.578970994477, 'ITALIE': 961.6109217497816, 'MAROC': 1887.577871162597, 'RWANDA': 6046.9665128774195, 'SENEGAL': 3874.3004595201833, 'GUATEMALA': 8934.06215679573, 'BELGIQUE': 493.34876529011234, 'AFRIQUE DU SUD': 8670.232492493116, 'REPUBLIQUE DOMINICAINE': 7139.943047514843, 'ISRAEL': 3324.760957431085, 'COLOMBIE': 8519.353047800576, 'PAYS-BAS': 681.1682112085996, 'POLOGNE': 5238.95537817839, 'CANADA': 6522.216063511657, 'COSTA RICA': 8946.180430280523, 'PORTUGAL': 1120.7411815824003, 'INDONÉSIE': 12172.18055459978, 'THAÏLANDE': 9700.993756294976, 'inde': 7245.67339386197, 'ROYAUME-UNI': 971.5166387817475, 'SRI LANKA': 8549.336373695536, 'EGYPTE': 3304.197561264577}


### Maintenant on a un dictionnaire qui lis les pays à leurs ditances par rapport à la france

In [53]:
distances_dict

{'ESPAGNE': 976.2843741584757,
 'FRANCE': 0.0,
 'PEROU': 9573.578970994477,
 'ITALIE': 961.6109217497816,
 'MAROC': 1887.577871162597,
 'RWANDA': 6046.9665128774195,
 'SENEGAL': 3874.3004595201833,
 'GUATEMALA': 8934.06215679573,
 'BELGIQUE': 493.34876529011234,
 'AFRIQUE DU SUD': 8670.232492493116,
 'REPUBLIQUE DOMINICAINE': 7139.943047514843,
 'ISRAEL': 3324.760957431085,
 'COLOMBIE': 8519.353047800576,
 'PAYS-BAS': 681.1682112085996,
 'POLOGNE': 5238.95537817839,
 'CANADA': 6522.216063511657,
 'COSTA RICA': 8946.180430280523,
 'PORTUGAL': 1120.7411815824003,
 'INDONÉSIE': 12172.18055459978,
 'THAÏLANDE': 9700.993756294976,
 'inde': 7245.67339386197,
 'ROYAUME-UNI': 971.5166387817475,
 'SRI LANKA': 8549.336373695536,
 'EGYPTE': 3304.197561264577}

In [55]:
# on l'associe à notre dataframe
df_legume["distance"] = df_legume["Origine"].map(distances_dict)

In [56]:
df_legume = df_legume[df_legume["Origine"].notnull()]   

In [57]:
#df_legume.to_csv("legumes_data_complete_carrefour.csv", index=False)

## Préprocessing données Agrybalise

### On trouve l'empreinte carbone d'un légume grâce au projet Agribalyse : https://doc.agribalyse.fr/documentation/utiliser-agribalyse/acces-donnees

### On a l'Impact détaillée par étapes du cycle de vie (Agriculture, Emballage, Transport,Supermarché et distribution, consommation)

In [58]:
df_empreinte = pd.read_excel("AGRIBALYSE3.1.1_produits alimentaires - detail etapes.xlsx", sheet_name="Detail etape", header=2)

In [59]:
df_empreinte["légume"] = df_empreinte["Nom du Produit en Français"].str.split(",").str[0]

In [60]:
df_empreinte["légume"].unique()

array(['Artichaut', 'Asperge', 'Aubergine', 'Avocat', 'Bambou',
       'Banane plantain', 'Bette ou blette', 'Betterave rouge', 'Brocoli',
       'Cardon', 'Carotte', 'Carottes râpées', 'Céleri branche',
       'Céleri-rave', 'Champignon', 'Chicorée rouge', 'Chicorée verte',
       'Chou blanc', 'Chou chinois (pak-choi ou pé-tsai)',
       'Chou chinois ou pak-choi ou pé-tsai', 'Chou de Bruxelles',
       'Chou frisé', 'Chou romanesco ou brocoli à pomme', 'Chou rouge',
       'Chou vert', 'Chou-fleur', 'Chou-rave', 'Citrouille',
       'Coeur de palmier', 'Concombre', 'Courge doubeurre (butternut)',
       'Courge hokkaïdo', 'Courge melonnette', 'Courge musquée',
       'Courge spaghetti', 'Courge', 'Courgette', 'Cresson alénois',
       'Cresson de fontaine', 'Crosne', 'Échalote', 'Endive', 'Épinard',
       'Fenouil', 'Fève à écosser', 'Fève', 'Fondue de poireau', 'Gombo',
       'Haricot beurre', 'Haricot blanc', 'Haricot de Lima',
       'Haricot flageolet', 'Haricot mungo germé ou

### On ne veut qu'une seule ligne pour un légume

In [61]:
df_empreinte = df_empreinte.groupby("légume").first().reset_index()
df_empreinte = df_empreinte[["légume", "Nom du Produit en Français","Agriculture","Transformation","Emballage","Transport","Supermarché et distribution","Consommation","Total" ]]
df_empreinte

,légume,Nom du Produit en Français,Agriculture,Transformation,Emballage,Transport,Supermarché et distribution,Consommation,Total
0,Artichaut,"Artichaut, appertisé, égoutté",0.648709,0.492404,0.286665,0.389983,0.026405,0.000000,1.844166
1,Asperge,"Asperge, appertisée, égouttée",0.532289,0.203049,0.171828,0.230945,0.015812,0.000000,1.153923
2,Aubergine,"Aubergine, crue",0.195727,0.000000,0.000000,0.240735,0.014917,0.005730,0.457109
3,Avocat,"Avocat, pulpe, cru",0.861392,0.000000,0.000000,0.463698,0.017811,0.137570,1.480470
4,Bambou,"Bambou, pousse, crue",0.702191,0.000000,0.000000,0.273563,0.018181,0.080351,1.074285
...,...,...,...,...,...,...,...,...,...
108,Tomate hors saison,"Tomate hors saison, crue",1.552315,0.000000,0.000000,0.308965,0.015591,0.080351,1.957221
109,Tomate verte,"Tomate verte, crue",0.314774,0.000000,0.000000,0.292678,0.015591,0.080351,0.703395
110,Tétragone cornue,"Tétragone cornue, cuite",0.137265,0.000000,0.231069,0.275093,0.044664,0.115593,0.803684
111,Échalote,"Échalote, crue",0.082227,0.000000,0.000000,0.240735,0.009761,0.032140,0.364863


In [62]:
df_empreinte_colonnes = ['légume', 'Nom du Produit en Français', 'Agriculture', 'Transformation',
       'Emballage', 'Transport', 'Supermarché et distribution', 'Consommation',
       'Total']

liste_legume = df_empreinte["légume"].str.lower().unique()


### Maintenant, nous avons les tablaux des légumes provenant de différentes origines et leur impact Co2 par étapes du cycle de vie, la prochaine tâche va être de fusionner les deux tableaux (scrapping et agribalyse) et d'ajouter des colonnes pour la presentation sur le site internet

In [ ]:
#on crée un colone légume, et pour chaques élément de la liste des légumes, on va chercher si le nom du légume est dans la colonne "Nom" de df_legume, si oui on va stocker le nom dans une nouvelle colonne "légume"
for legume in liste_legume:
    df_legume[legume] = df_legume["Nom"].str.lower().str.contains(legume)


In [ ]:
#si une colonne contient sur toutes les lignes des False, on la supprime
df_legume = df_legume.loc[:, (df_legume != False).any(axis=0)]

In [67]:
import numpy as np
df_legume["légume"] = np.nan

liste_legume = ['artichaut', 'asperge', 'aubergine',
       'avocat', 'brocoli', 'carotte', 'champignon', 'chou blanc',
       'chou de bruxelles', 'chou rouge', 'chou vert', 'chou-fleur',
       'citrouille', 'concombre', 'courge', 'courge spaghetti', 'courgette',
       'céleri branche', 'endive', 'fenouil', 'haricot mungo', 'haricot vert',
       'haricots verts', 'laitue', 'laitue iceberg', 'laitue romaine',
       'mâche', 'navet', 'oignon', 'panais', 'piment', 'poireau',
       'poivron', 'poivron rouge', 'poivron vert', 'potimarron', 'potiron',
       'radis noir', 'radis rouge', 'roquette', 'rutabaga', 'scarole',
       'tomate', 'tomate cerise', 'épinard']

for legume in liste_legume:
    df_legume.loc[df_legume[legume] == True, "légume"] = legume


df_legume.drop(liste_legume, axis=1, inplace=True)

df_legume

/var/folders/kq/kcblb0_91wn66p2h_d11w7l80000gn/T/ipykernel_13312/695877785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_legume["légume"] = np.nan
/var/folders/kq/kcblb0_91wn66p2h_d11w7l80000gn/T/ipykernel_13312/695877785.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_legume.drop(liste_legume, axis=1, inplace=True)


,lien du produit,Nom,Image,Origine,Prix,Prix à l'unité,Poids,nom_legume,distance,légumes,légume
0,https://www.carrefour.fr/p/avocat-bio-carrefou...,Avocat Bio CARREFOUR BIO,https://media.carrefour.fr/medias/c7d433b8d8b4...,ESPAGNE,"1,69",1.69 € / U,NaN,Avocat,976.284374,False,avocat
1,https://www.carrefour.fr/p/avocat-hass-bio-352...,Avocat Hass Bio,https://media.carrefour.fr/medias/fbf517249c36...,ESPAGNE,"1,69",1.69 € / U,NaN,Avocat,976.284374,False,avocat
2,https://www.carrefour.fr/p/avocat-300000103757...,Avocat,https://media.carrefour.fr/medias/5e7a6105d1ec...,ISRAEL,"1,19",1.19 € / U,NaN,Avocat,3324.760957,False,avocat
3,https://www.carrefour.fr/p/asperges-violette-b...,Asperges violette Bio CARREFOUR BIO,https://media.carrefour.fr/medias/fdea62a7ce86...,FRANCE,"6,99",13.98 € / KG,NaN,Asperges,0.000000,False,asperge
4,https://www.carrefour.fr/p/asperges-verte-bio-...,Asperges verte Bio CARREFOUR BIO,https://media.carrefour.fr/medias/12c2e494c77c...,FRANCE,"6,99",17.48 € / KG,NaN,Asperges,0.000000,False,asperge
...,...,...,...,...,...,...,...,...,...,...,...
629,https://www.carrefour.fr/p/curry-feuilles-indi...,Curry Feuilles Indiennes 20g Kari Patta,https://media.carrefour.fr/medias/2b985aed02c2...,SRI LANKA,"5,45",NaN,NaN,Curry,8549.336374,False,NaN
630,https://www.carrefour.fr/p/curry-indien-melang...,Curry indien mélange d'épices doux 100g,https://media.carrefour.fr/medias/733c96661c47...,inde,"2,07",NaN,NaN,Curry,NaN,False,NaN
631,https://www.carrefour.fr/p/piment-rouge-doux-e...,Piment Rouge Doux En Poudre Deggi Mirch 100g,https://media.carrefour.fr/medias/5ba3cbaca451...,inde,"3,50",NaN,NaN,Piment,NaN,False,piment
633,https://www.carrefour.fr/p/piment-doux-en-poud...,Piment Doux En Poudre Quintessence 100g,https://media.carrefour.fr/medias/bd071fb10315...,ESPAGNE,"4,15",NaN,NaN,Piment,976.284374,False,piment


In [ ]:
df_legume.dropna(subset=["légume"], inplace=True)

In [71]:
df_empreinte["légume"] = df_empreinte["légume"].str.lower()

## Fusion des deux tables

In [72]:
#on fusionne les deux dataframes
df_legume_empreinte = pd.merge(df_legume, df_empreinte, on="légume", how="left")
df_legume_empreinte

#rename la colonne Total en Total empreinte en kg CO2 par kg de produit pour plus de précision

df_legume_empreinte.rename(columns={"Total": "Total empreinte en kg CO2 par kg de produit"}, inplace=True)

In [73]:
df_legume_empreinte

,lien du produit,Nom,Image,Origine,Prix,Prix à l'unité,Poids,nom_legume,distance,légumes,légume,Nom du Produit en Français,Agriculture,Transformation,Emballage,Transport,Supermarché et distribution,Consommation,Total empreinte en kg CO2 par kg de produit
0,https://www.carrefour.fr/p/avocat-bio-carrefou...,Avocat Bio CARREFOUR BIO,https://media.carrefour.fr/medias/c7d433b8d8b4...,ESPAGNE,"1,69",1.69 € / U,NaN,Avocat,976.284374,False,avocat,"Avocat, pulpe, cru",0.861392,0.000000,0.000000,0.463698,0.017811,0.137570,1.480470
1,https://www.carrefour.fr/p/avocat-hass-bio-352...,Avocat Hass Bio,https://media.carrefour.fr/medias/fbf517249c36...,ESPAGNE,"1,69",1.69 € / U,NaN,Avocat,976.284374,False,avocat,"Avocat, pulpe, cru",0.861392,0.000000,0.000000,0.463698,0.017811,0.137570,1.480470
2,https://www.carrefour.fr/p/avocat-300000103757...,Avocat,https://media.carrefour.fr/medias/5e7a6105d1ec...,ISRAEL,"1,19",1.19 € / U,NaN,Avocat,3324.760957,False,avocat,"Avocat, pulpe, cru",0.861392,0.000000,0.000000,0.463698,0.017811,0.137570,1.480470
3,https://www.carrefour.fr/p/asperges-violette-b...,Asperges violette Bio CARREFOUR BIO,https://media.carrefour.fr/medias/fdea62a7ce86...,FRANCE,"6,99",13.98 € / KG,NaN,Asperges,0.000000,False,asperge,"Asperge, appertisée, égouttée",0.532289,0.203049,0.171828,0.230945,0.015812,0.000000,1.153923
4,https://www.carrefour.fr/p/asperges-verte-bio-...,Asperges verte Bio CARREFOUR BIO,https://media.carrefour.fr/medias/12c2e494c77c...,FRANCE,"6,99",17.48 € / KG,NaN,Asperges,0.000000,False,asperge,"Asperge, appertisée, égouttée",0.532289,0.203049,0.171828,0.230945,0.015812,0.000000,1.153923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,https://www.carrefour.fr/p/ail-oignons-echalot...,Ail / Oignons / Echalotes : Panier De Condime...,https://media.carrefour.fr/medias/61c7c29d4d82...,FRANCE,"24,95",8.32 € / kg,NaN,Ail,0.000000,False,oignon,"Oignon, cru",0.100028,0.000000,0.000000,0.246355,0.011013,0.032140,0.389536
359,https://www.carrefour.fr/p/oignons-rouges-de-t...,Oignons Rouges De Tropea Igp Aigre-doux 280 Gr,https://media.carrefour.fr/medias/35e8fa71fa98...,ITALIE,"5,09",NaN,NaN,Oignons,961.610922,False,oignon,"Oignon, cru",0.100028,0.000000,0.000000,0.246355,0.011013,0.032140,0.389536
360,https://www.carrefour.fr/p/oignon-rouge-carame...,Oignon Rouge Caramélisée 150 Gr,https://media.carrefour.fr/medias/a5599306fe13...,ITALIE,"5,59",NaN,NaN,Oignon,961.610922,False,oignon,"Oignon, cru",0.100028,0.000000,0.000000,0.246355,0.011013,0.032140,0.389536
361,https://www.carrefour.fr/p/piment-rouge-doux-e...,Piment Rouge Doux En Poudre Deggi Mirch 100g,https://media.carrefour.fr/medias/5ba3cbaca451...,inde,"3,50",NaN,NaN,Piment,NaN,False,piment,"Piment, cru",0.756058,0.000000,0.000000,0.325927,0.021284,0.080351,1.183620


In [ ]:
# on drop les distance nan
df_legume_empreinte.dropna(subset=["distance"], inplace=True)


In [76]:
#on créé un nouveau dataframe contenant uniquement les colonnes légume, distance et origine et on drop les doublons
df_legume_empreinte_ref = df_legume_empreinte[["légume", "distance", "Origine"]].drop_duplicates()
df_legume_empreinte_ref


### Création de colonnes pour le website

In [ ]:

# on trie par légume et distance et on on crée une colonne "rang" qui contient le rang de la distance par légume
df_legume_empreinte_ref["rang"] = df_legume_empreinte_ref.groupby("légume")["distance"].rank()
df_legume_empreinte_ref

#on créé une colonne couleur transport qui contient la couleur en fonction du rang
df_legume_empreinte_ref["couleur transport"] = df_legume_empreinte_ref["rang"].apply(lambda x: "green" if x == 1.0 else "jaune" if x == 2.0 else "orange" if x == 3.0 else "red")



In [77]:
df_legume_empreinte_ref

,légume,distance,Origine,rang,couleur transport
0,avocat,976.284374,ESPAGNE,1.0,green
2,avocat,3324.760957,ISRAEL,2.0,jaune
3,asperge,0.000000,FRANCE,1.0,green
8,asperge,976.284374,ESPAGNE,2.0,jaune
10,asperge,9573.578971,PEROU,3.0,orange
...,...,...,...,...,...
291,navet,0.000000,FRANCE,1.0,green
303,oignon,0.000000,FRANCE,1.0,green
344,artichaut,961.610922,ITALIE,2.0,jaune
347,piment,961.610922,ITALIE,2.0,jaune


### On veut maintenant prendre la distance en compte dans l'empreinte carbone, pour cela, on regarde si le légume à plusieurs origine possibles, si c'est le cas, on ajoute un malus à l'empreinte carbone aux légumes qui ont les orignes les plus loins : *1,15 pour la deuxième origine la plus proche, *1,3 pour la troisième origine la plus proche.

In [ ]:
# on crée une nouvelle colonne distance_pourcentage qui est 0 si le rang est 1, 0.15 si le rang est 2, 0.3 si le rang est 3 et 0.4 si le rang est 4
df_legume_empreinte_ref["distance_pourcentage"] = df_legume_empreinte_ref["rang"].apply(lambda x: 1 if x == 1.0 else 1.15 if x == 2.0 else 1.3 if x == 3.0 else 1.4)


In [ ]:
#on merge df_legume_empreinte_ref avec df_legume_empreinte
df_legume_empreinte = pd.merge(df_legume_empreinte, df_legume_empreinte_ref, on=["légume", "distance", "Origine"], how="left")

#on multiplie la colonne "Transport" par la colonne "distance_pourcentage" et on recalcul le "Total empreinte en kg CO2 par kg de produit" qui est la somme de Agriculture	Transformation	Emballage	Transport	Supermarché et distribution	Consommation
df_legume_empreinte["Transport"] = df_legume_empreinte["Transport"] * df_legume_empreinte["distance_pourcentage"]
df_legume_empreinte["Total empreinte en kg CO2 par kg de produit"] = df_legume_empreinte["Agriculture"] + df_legume_empreinte["Transformation"] + df_legume_empreinte["Emballage"] + df_legume_empreinte["Transport"] + df_legume_empreinte["Supermarché et distribution"] + df_legume_empreinte["Consommation"]



### Export de la nouvelle table

In [80]:
df_legume_empreinte.to_csv("legumes_data_complete_carrefour_avec_empreinte_2.csv", index=False)